Before running this notebook, make sure you've downloaded road curvature data and unzipped the KML file. See the README on how to do this!
This notebook is designed to allow you to interrogate the data and see what is going on underneath the hood! I encourage you to plot and visualise the data at each stage of the calculations!

We first start by setting up our map data. We use a 'graph' to represent the map (i.e.: 'nodes' are road crossing, and every road is an 'edge'). The parse_data script defines this graph by downloading data for an area of a chosen size from your desired starting point. It also reads the KML file (which contains information about the road's windiness) and joins this onto the edge information - we will use this later to define what the best routes are.

For example, let's assume we want to take a drive around the Peak District. We have driven to Parsley Hay car park and want to use this as our starting point. We define:
- the starting (center_point) as coordinates. Use Google Maps to find this.
- the size of the map region to download. As a rule of thumb, make this about half of the distance you'd look to drive. You can make this bigger but it will take longer to compute the graph!

In [ ]:
import parse_data

center_point = (53.169910, -1.780962)
dist_meters = 15000
network_type = 'drive'
kml_path = 'data/great-britain.c_300.kml'
save_path = 'data/'

nodes, edges = parse_data.fetch_roads(center_point, dist_meters, network_type) # this can take a couple of minutes depending on how dense the road network you're looking at is
curvature_gdf = parse_data.parse_kml_curvature(kml_path)
enriched_edges = parse_data.join_curvature(edges, curvature_gdf)
parse_data.save_outputs(enriched_edges, nodes.copy(), save_path)

We now have a list of nodes and edges to compose the graph from in the pathfinder script. If you're curious, you can use the code below to visualise the resulting network of roads and the node and edge GeoDataframe.

In [ ]:
import osmnx as ox
G = ox.graph_from_gdfs(nodes, edges)
fig, ax = ox.plot_graph(G, node_size=5, edge_color='gray', edge_linewidth=0.5)

print(nodes.head(5))
print(edges.head(5))

Now that we have all our data ready, we can start planning our route!

We first start by plotting finding our starting point and using a modified depth limited search algorithm to search find good candidates for the first half of our journey (the outbound part).

In [ ]:
import path_finder

target_distance = 30000
start_lat = center_point[0]
start_lon = center_point[1]
output = 'data/output/'

start_node = ox.nearest_nodes(G, start_lon, start_lat)

outbound_paths = path_finder.calculate_outbound_paths(G, start_node, target_distance=(target_distance)/2, max_paths=1500, max_depth=30)
print('   ', {len(outbound_paths)}, 'paths found')


We can now score each path. The scoring method is based on several factors:
- how windy the route is
- the average speed limit along the route
- the number of unique roads used
These scores are normalised (such that a road can have a windiness score from 0-1 for example), and a factor of 2 is applied to the windiness score (as this is the factor we care most about)
The result is an overall score that can vary between -1 and 4.

In [ ]:
print('>> Scoring outbound paths...')
outbound_paths = path_finder.calculate_scores(G, outbound_paths, start_lon, start_lat, nodes=nodes)

At this point, we have a lot of routes that are very similar (this is a biproduct of the depth limited search algorithm). We will need to reduce the number of possible paths because we wouldn't want to calculate the return routes for all of them (this would take ages). This would also be quite wasteful as if the outbound routes are similar, the return routes will also be similar.
If routes are more than 50% similar (i.e.: half of the roads used for a route are the same as another's), we remove these and only keep the route with the highest score.

In [ ]:
print('>> Removing similar paths...')
filtered_outbound_paths = path_finder.remove_similar_paths(outbound_paths, similarity_threshold=0.5)

Now that we have a more manageable number of routes to consider, we want to cluster them into groups based on the bearing they are relative to the starting point. This may seem odd at first, but we do this to ensure that we have a good variety of route choice available to us. If the main goal is to explore the area, we wouldn't want to limit ourselves to one direction only, even if that has all the best roads!

To do this, we take the 'centroid' for each route (i.e.: just the average of all the node coordinates) and calculate the bearing of this relative to the start point. Then, we use KMeans clustering to group the routes and select the 5 best routes from each group. This will loosely equate to the best routes in each of the 4 cardinal directions.

In [ ]:
print('>> Clustering results and selecting best outbound options...')
clustered_outbound_paths = path_finder.cluster_paths(filtered_outbound_paths, n_clusters=4)
best_outbound_paths = path_finder.select_top_paths_by_cluster(clustered_outbound_paths, top_n=5)


A cool plot that can show what is happening here is shown below:

In [ ]:
import matplotlib.pyplot as plt
def polar_plot(bearing_vector, score_vector, colour_key):
    label_to_color = {
    0: 'red',
    1: 'blue',
    2: 'green',
    3: 'orange'
    }
    colors = [label_to_color[label] for label in colour_key]

    # Create polar plot using scatter
    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    ax.scatter(bearing_vector, score_vector, c=colors)

    plt.show()

polar_plot(best_outbound_paths['bearing'].to_list(), best_outbound_paths['score'].to_list(), best_outbound_paths['labels'].to_list())

With the best outbound routes chosen, we can start finding our way back to the start point. We will do this still by prioritising windy roads, and also try to avoid doubling back on ourselves and using the same roads we used to come out.
Once this done we can recalculate all the scores and take the 5 best routes out of all the options!

In [ ]:
print('>> Plotting return journeys for best outbound path candidates...')
completed_paths = path_finder.create_full_loops(G, best_outbound_paths, start_node)
print('>> Calculating full journey scores and saving best options...')
completed_paths = path_finder.calculate_scores(G, completed_paths['path_nodes'].to_list(), start_lon, start_lat, nodes=nodes)
proposed_paths = completed_paths.sort_values(by='score', ascending=False).head(5)
path_finder.plot_outputs(G, proposed_paths, start_lat, start_lon, output)
path_finder.save_output(proposed_paths, output)
print('>> Results complete. View map in the results.html, and see route metrics in results.csv files. These have been saved to the output directory of your choice.')
